# Eniac — the e-commerce

## So our task in this noutbook is:

__With Orderlines_df:__
* drop product_id - all nulls
* convert unit_price to float
* convert date from object to date

__Orders_df tasks:__
* convert date from object to date
* delete all null values in total_paid columns

__Products_df tasks:__
* drop 8746 duplicates
* solve problem with additional dots in price and promo_price columns
* convert price and promo_price
* deside what to do with in_stock columns



In [129]:
import os
import pandas as pd 
import numpy as np
from utils import *

In [100]:
brands_df = pd.read_csv("./Datasets/brands.csv")
orderlines_df = pd.read_csv("./Datasets/orderlines.csv")
orders_df = pd.read_csv("./Datasets/orders.csv")
products_df = pd.read_csv("./Datasets/products.csv")

## Orderlines_df

In [101]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [102]:
orderlines_df = orderlines_df.drop(columns="product_id")

In [103]:
orderlines_df.sample(10)

,id,id_order,product_quantity,sku,unit_price,date
269605,1611020,510345,1,APP2478,793.00,2018-02-12 20:24:50
262335,1599627,505654,1,APP0662,159.00,2018-02-04 19:16:10
291357,1645665,525230,1,APP1175,626.00,2018-03-10 22:17:25
173261,1456296,447469,1,BEL0371,23.99,2017-11-27 10:45:14
163010,1439246,440106,1,APP2244,2.411.59,2017-11-24 16:42:22
84174,1281010,371938,1,APP1190,56.00,2017-07-03 14:08:53
252715,1584528,500088,1,APP1083,219.00,2018-01-28 12:47:17
62423,1242926,353391,1,APP1221,109.00,2017-05-07 07:18:57
65392,1248054,355939,1,CRU0016-2,87.99,2017-05-15 13:05:16
19540,1163546,316754,1,LOG0084,19.99,2017-01-29 19:52:30


In [104]:
# as we can see, we found problems with additional dots in products_d,f so check it here
check_dots(orderlines_df, "unit_price")

unit_price
5     138392
6     104406
8      36136
4      15015
9         32
10         1
7          1
Name: count, dtype: int64


In [105]:
# correct these numbers

to_remove_extra_dots(orderlines_df, "unit_price")

********
Have Done
********


In [106]:
orderlines_df.describe()

,id,id_order,product_quantity,unit_price
count,2.939830e+05,293983.000000,293983.000000,293983.000000
mean,1.397918e+06,419999.116544,1.121126,409.855192
std,1.530096e+05,66344.486479,3.396569,837.065461
min,1.119109e+06,241319.000000,1.000000,-119.000000
25%,1.262542e+06,362258.500000,1.000000,35.990000
50%,1.406940e+06,425956.000000,1.000000,92.990000
75%,1.531322e+06,478657.000000,1.000000,339.580000
max,1.650203e+06,527401.000000,999.000000,159989.830000


As we can see, after the corrections, we received a bit of weird outliers with one negative price, and one price that abnormally high. So let`s check these prices and correct or drop them.

In [107]:
min_mask = (orderlines_df['unit_price'] < 0) 
max_mask = (orderlines_df['unit_price'] == orderlines_df['unit_price'].max())
min_value = orderlines_df.loc[min_mask] 
max_value = orderlines_df.loc[max_mask, :]

print(min_value, '\n**\n ', max_value)

            id  id_order  product_quantity      sku  unit_price  \
77008  1268645    365886                 1  APP1465      -119.0   

                      date  
77008  2017-06-15 12:48:54   
**
              id  id_order  product_quantity      sku  unit_price  \
36979  1197439    331780                 1  NEA0009   159989.83   

                      date  
36979  2017-03-03 22:29:21  


In [108]:
#to check min and max price in order df and products df
## min
print(min_value)
print(products_df.loc[products_df['sku'] == min_value.iloc[0,4]])
print()
print(orders_df.loc[orders_df['order_id'] == min_value.iloc[0,1]])

            id  id_order  product_quantity      sku  unit_price  \
77008  1268645    365886                 1  APP1465      -119.0   

                      date  
77008  2017-06-15 12:48:54  
Empty DataFrame
Columns: [sku, name, desc, price, promo_price, in_stock, type]
Index: []

       order_id         created_date  total_paid      state
66340    365886  2017-06-15 12:51:18        30.0  Cancelled


As we can see this order was Cancelled, we can drop it

In [109]:
orderlines_df = orderlines_df.drop(min_value.index, axis=0)
orderlines_df.describe()

,id,id_order,product_quantity,unit_price
count,2.939820e+05,293982.000000,293982.000000,293982.000000
mean,1.397919e+06,419999.300614,1.121126,409.856991
std,1.530096e+05,66344.524250,3.396574,837.066316
min,1.119109e+06,241319.000000,1.000000,0.000000
25%,1.262542e+06,362258.250000,1.000000,35.990000
50%,1.406941e+06,425956.500000,1.000000,92.990000
75%,1.531322e+06,478657.000000,1.000000,339.580000
max,1.650203e+06,527401.000000,999.000000,159989.830000


In [110]:
##max
print(max_value)
print(products_df.loc[products_df['sku'] == max_value.iloc[0,4]])
print(orders_df.loc[orders_df['order_id'] == max_value.iloc[0,1]])

            id  id_order  product_quantity      sku  unit_price  \
36979  1197439    331780                 1  NEA0009   159989.83   

                      date  
36979  2017-03-03 22:29:21  
Empty DataFrame
Columns: [sku, name, desc, price, promo_price, in_stock, type]
Index: []
       order_id         created_date  total_paid            state
32296    331780  2017-03-03 22:29:21   159989.83  Shopping Basket


The same procedure for the max value

In [111]:
# to normalise Date and final checks

orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])


print(f"Duplicated: {orderlines_df.duplicated().sum()}")
print(f"Null values:\n{orderlines_df.isna().sum()}")

orderlines_df.info()
orderlines_df.sample(20)

Duplicated: 0
Null values:
id                  0
id_order            0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 293982 entries, 0 to 293982
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293982 non-null  int64         
 1   id_order          293982 non-null  int64         
 2   product_quantity  293982 non-null  int64         
 3   sku               293982 non-null  object        
 4   unit_price        293982 non-null  float64       
 5   date              293982 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 15.7+ MB


,id,id_order,product_quantity,sku,unit_price,date
143735,1400862,423573,1,OWC0153,49.99,2017-11-08 19:09:49
15500,1155357,313298,2,CRU0023,44.99,2017-01-22 18:45:46
45398,1213046,338989,1,APP0995,2068.99,2017-03-25 11:39:53
3120,1129691,302392,1,SPE0168,19.99,2017-01-04 12:01:56
36486,1196505,331361,1,IFX0045,29.90,2017-03-02 17:41:00
47453,1216877,340733,1,WAC0220,304.99,2017-03-30 10:26:11
116835,1352570,400114,1,APP1177,627.33,2017-09-14 17:08:30
91376,1294048,378089,1,PAC0776,113.99,2017-07-16 11:38:54
187221,1475903,454944,1,QNA0256,336.99,2017-12-04 11:51:28
127084,1371231,409180,1,APP1214,82.99,2017-10-06 02:46:09


## Orders_df

In [112]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [113]:
print(orders_df.duplicated().sum())
print(orders_df['total_paid'].isna().value_counts())

0
total_paid
False    226904
True          5
Name: count, dtype: int64


In [114]:
orders_df = orders_df.dropna(axis=0)
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


## Products_df

In [115]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


In [116]:
products_df = products_df.dropna()
products_df= products_df.drop_duplicates()
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392


In [117]:
products_df.info()
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10477 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10477 non-null  object
 1   name         10477 non-null  object
 2   desc         10477 non-null  object
 3   price        10477 non-null  object
 4   promo_price  10477 non-null  object
 5   in_stock     10477 non-null  int64 
 6   type         10477 non-null  object
dtypes: int64(1), object(6)
memory usage: 654.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 293982 entries, 0 to 293982
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293982 non-null  int64         
 1   id_order          293982 non-null  int64         
 2   product_quantity  293982 non-null  int64         
 3   sku               293982 non-null  object        
 4   unit_price        293982 non-null  f

In [118]:
check_dots(products_df, "price")
to_remove_extra_dots(products_df, "price")

price
5     3589
6     2089
4     1620
3     1084
2      918
7      795
9      225
10     138
11      10
1        8
8        1
Name: count, dtype: int64
********
Have Done
********


In [119]:
orderlines_df_un = orderlines_df.groupby("sku").agg({"unit_price":"mean"})
orderlines_df_un = orderlines_df_un.reset_index()
orderlines_df_un

,sku,unit_price
0,8MO0001-A,12.960000
1,8MO0003-A,12.850000
2,8MO0007,21.369310
3,8MO0008,21.723333
4,8MO0009,17.842857
...,...,...
7946,ZAG0041,19.990000
7947,ZAG0042,19.990000
7948,ZEP0007,137.118000
7949,ZEP0008,104.120000


In [ ]:
# from utils import price_correction #one more way to solve this problem

products_df_cp_clear_price = (
    products_df.loc[:, ['sku','name','desc', 'price',
                           'promo_price', 'in_stock', 'type']].
                           merge(
                              orderlines_df_un.loc[:, ['sku','unit_price']],
                              on='sku',
                              how='inner'
))
products_df_cp_clear_price.shape

# products_df_cp_clear_price['price_correction'] = products_df_cp_clear_price.apply(price_correction, axis=1) #one more way to solve this problem



(7807, 8)

In [ ]:
diff_mask = products_df_cp_clear_price["price"]/products_df_cp_clear_price["unit_price"] > 2.6
products_df_cp_clear_price.loc[diff_mask]

,sku,name,desc,price,promo_price,in_stock,type,unit_price
79,ROL0010,Rollei Chest Mount iPhone 4 / 4S,Case iPhone 4 / 4S for sport.,49.990,99.898,0,11865403,9.990000
110,NTE0023,NewerTech eSATA Cable Adapter Mac Pro,Add 2 eSATA ports with the adapter cable for M...,30.990,73.895,1,12755395,7.118571
204,PIE0011,Internal Battery for iPhone 3G,Replacement AC Adapter for Apple iPhone 3G.,98.978,98.978,0,21485407,9.555000
237,SEV0026,Service installation RAM + HDD + SSD MacBook /...,RAM + HDD installation + SSD in your MacBook /...,599.918,599.918,0,20642062,58.097853
244,SEV0024,Service installation RAM + HDD + SSD Mac mini,installation RAM HDD + SSD + on your Mac mini ...,599.918,599.918,0,20642062,57.800993
...,...,...,...,...,...,...,...,...
7749,UBI0007,Ubiquiti Amplifi Wi-Fi Mesh Router + 2 Mesh Ac...,Wi-Fi high-density intelligent Mesh technology,35998.952,3.599.895,0,1334,359.990000
7751,UBI0008,Ubiquiti Amplifi Wi-Fi Mesh Router,Smart Wi-Fi Router high density Mesh technology,1299.891,1.299.891,0,1334,129.990000
7779,OWC0262-A,"Open - OWC Express External box 25 ""USB 3.0 Plata",Outer carton for HDDs and SDD format 25 inches...,36.990,13.001,0,11935397,13.000000
7785,WAC0248,Wacom support for MobileStudio Pro,Wacom support MobileStudio Pro 13 inches and 1...,9990.002,99.9,0,101781405,99.900000


In [ ]:
import numpy as np

conditions = [
    products_df_cp_clear_price["unit_price"] == 0,
    products_df_cp_clear_price["price"] / products_df_cp_clear_price["unit_price"] < 2.6
]

choises = [
    products_df_cp_clear_price["price"],
    products_df_cp_clear_price["price"]
]

products_df_cp_clear_price["price_corrections"] = np.select(conditions, choises, default=products_df_cp_clear_price["price"]/10)

In [123]:
products_df_cp_clear_price

,sku,name,desc,price,promo_price,in_stock,type,unit_price,price_corrections
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,46.276239,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,589.996,0,13855401,55.377605,59.00
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,569.898,0,1387,57.073226,59.00
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,229.997,0,1230,23.731739,25.00
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,34.420000,34.99
...,...,...,...,...,...,...,...,...,...
7802,MMW0012,"My MW Case MacBook Pro 13 ""(Late 2016) Blue Sa...",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,0,13835403,21.840000,29.99
7803,MMW0013,"My MW Case MacBook Pro 13 ""(Late 2016) White S...",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,22.990000,29.99
7804,MMW0016,"My MW Case MacBook Pro 13 ""(Late 2016) Black",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,22.277073,29.99
7805,MMW0015,"My MW Case MacBook Pro 13 ""(Late 2016) Gray",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,22.263684,29.99


In [124]:
diff_mask_ = products_df_cp_clear_price["price_corrections"]/products_df_cp_clear_price["unit_price"] > 2.6
products_df_cp_clear_price = products_df_cp_clear_price.loc[~diff_mask_]
products_df_cp_clear_price

,sku,name,desc,price,promo_price,in_stock,type,unit_price,price_corrections
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,46.276239,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,589.996,0,13855401,55.377605,59.00
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,569.898,0,1387,57.073226,59.00
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,229.997,0,1230,23.731739,25.00
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,34.420000,34.99
...,...,...,...,...,...,...,...,...,...
7802,MMW0012,"My MW Case MacBook Pro 13 ""(Late 2016) Blue Sa...",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,0,13835403,21.840000,29.99
7803,MMW0013,"My MW Case MacBook Pro 13 ""(Late 2016) White S...",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,22.990000,29.99
7804,MMW0016,"My MW Case MacBook Pro 13 ""(Late 2016) Black",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,22.277073,29.99
7805,MMW0015,"My MW Case MacBook Pro 13 ""(Late 2016) Gray",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,22.263684,29.99


In [125]:
products_df_copy = products_df_cp_clear_price.drop(columns=["price","promo_price","unit_price"])


In [126]:
products_df_copy = products_df_copy.rename(columns={"price_corrections":"price"})
products_df_copy

,sku,name,desc,in_stock,type,price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,1,8696,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,0,13855401,59.00
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,0,1387,59.00
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,0,1230,25.00
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,1,1364,34.99
...,...,...,...,...,...,...
7802,MMW0012,"My MW Case MacBook Pro 13 ""(Late 2016) Blue Sa...",Avoid shock and damage to your MacBook Pro 13-...,0,13835403,29.99
7803,MMW0013,"My MW Case MacBook Pro 13 ""(Late 2016) White S...",Avoid shock and damage to your MacBook Pro 13-...,1,13835403,29.99
7804,MMW0016,"My MW Case MacBook Pro 13 ""(Late 2016) Black",Avoid shock and damage to your MacBook Pro 13-...,1,13835403,29.99
7805,MMW0015,"My MW Case MacBook Pro 13 ""(Late 2016) Gray",Avoid shock and damage to your MacBook Pro 13-...,1,13835403,29.99


In [ ]:
products_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7756 entries, 0 to 7806
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       7756 non-null   object 
 1   name      7756 non-null   object 
 2   desc      7756 non-null   object 
 3   in_stock  7756 non-null   int64  
 4   type      7756 non-null   object 
 5   price     7756 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 424.2+ KB


In [131]:
local_path = "./Datasets/Clean/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
    
brands_df.to_csv("./Datasets/Clean/brands.csv")
orderlines_df.to_csv("./Datasets/Clean/orderlines.csv")
orders_df.to_csv("./Datasets/Clean/orders.csv")
products_df_copy.to_csv("./Datasets/Clean/products.csv")